In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, PowerTransformer
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:

# Membaca data dari file CSV
df = pd.read_csv('gldm_features_Data_Enhance.csv')

# Membagi data menjadi beberapa dataframe berdasarkan arah
directions = ['0', '45', '90', '135']
dataframes = []

for direction in directions:
    cols = [f'Contrast_{direction}', f'ASM_{direction}', f'Entropy_{direction}', f'Mean_{direction}',
            f'IDM_{direction}']
    df_direction = df[['Image'] + cols]
    df_direction['Label'] = df.index.map(lambda x: 0 if x <= 600 else 1)
    dataframes.append(df_direction)



C:\Users\hanif\AppData\Local\Temp\ipykernel_10780\3336776921.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_direction['Label'] = df.index.map(lambda x: 0 if x <= 600 else 1)
C:\Users\hanif\AppData\Local\Temp\ipykernel_10780\3336776921.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_direction['Label'] = df.index.map(lambda x: 0 if x <= 600 else 1)
C:\Users\hanif\AppData\Local\Temp\ipykernel_10780\3336776921.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

AttributeError: 'list' object has no attribute 'columns'

In [ ]:
# Fungsi untuk melatih dan menguji model SVM dengan Grid Search
def train_test_SVM(X_train, y_train, X_test, y_test):
    scaler = PowerTransformer()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Daftar kombinasi hiperparameter yang akan diuji
    param_grid = {'C': [1, 10, 100],
                  'gamma': [0.1, 0.01, 0.001],
                  'kernel': ['rbf', 'linear', 'poly']}
    
    svm = SVC()

    # Mencari kombinasi hiperparameter terbaik menggunakan Grid Search
    grid_search = GridSearchCV(svm, param_grid, cv=5)
    grid_search.fit(X_train_scaled, y_train)

    # Menggunakan model terbaik untuk prediksi
    best_svm = grid_search.best_estimator_
    y_pred = best_svm.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    
    return accuracy, best_svm


# List untuk menyimpan akurasi per parameter dan direction
accuracies = []

# Daftar parameter yang akan diuji
parameters = ['Contrast', 'ASM', 'Entropy', 'Mean', 'IDM']

# Melakukan train dan test untuk setiap parameter dan direction
for parameter in parameters:
    for i, df_direction in enumerate(dataframes):
        # Memilih kolom dataframe sesuai parameter
        cols = [col for col in df_direction.columns if parameter in col]
        df_parameter_direction = df_direction[['Image', 'Label'] + cols]
        
        X = df_parameter_direction.drop(['Image', 'Label'], axis=1)
        y = df_parameter_direction['Label']
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        accuracy, best_svm = train_test_SVM(X_train, y_train, X_test, y_test)
        accuracies.append(accuracy)
        
        print(f"Akurasi {parameter} Direction {directions[i]}")
        #print(df_parameter_direction)
        
        print(f"Accuracy: {accuracy}")
        print("Best SVM Model:", best_svm)
        print()

In [ ]:
# Membuat DataFrame kosong
results_df = pd.DataFrame(columns=['Parameter', '0', '45', '90', '135', 'Average'])

# Daftar parameter yang akan diuji
parameters = ['Contrast', 'ASM', 'Entropy', 'Mean', 'IDM']

# Melakukan iterasi untuk setiap parameter
for i, parameter in enumerate(parameters):
    # Menambahkan baris baru ke DataFrame
    results_df.loc[i] = [parameter] + accuracies[i*4:(i+1)*4] + [sum(accuracies[i*4:(i+1)*4])/4]

# Menampilkan DataFrame hasil
print(results_df)